# Role R Likelihood-free Simulation Based Inference

This is for multi-core simulation and GPU training and inference.

In [1]:
from roler.model import ModelPrior
from roler.distributions import *
from roler.simulation import Simulator

prior = ModelPrior(
    individuals_local=IntDistribution(50, 300),
    individuals_meta=IntDistribution(400, 1000),
    species_meta=50,
    speciation_local=0.05,
    speciation_meta=0.05,
    extinction_meta=0.05,
    env_sigma=0.5,
    trait_sigma=1,
    comp_sigma=0.5,
    dispersal_prob=0.1,
    mutation_rate=0.01,
    equilib_escape=1,
    num_basepairs=250,
    init_type='oceanic_island',
    niter=2000,
    niterTimestep=10
)

simulator = Simulator()

/home/micahtilton/github/likelihood-free-inference/.venv/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


Installing the 'roleR' R package from GitHub...


R[write to console]: Skipping install of 'roleR' from a github remote, the SHA1 (cc6546a1) has not changed since last install.
  Use `force = TRUE` to force installation



In [7]:
from roler.datasets import Dataset

dataset = Dataset(simulator=simulator, prior=prior)
x, y = dataset.generate_dataset(16*100, select_last_n=10, n_jobs=14)

In [8]:
print(x.shape)
print(y.shape)

torch.Size([16000, 2])
torch.Size([16000, 9])


In [9]:
from sbi.inference import SNPE

device = "cuda"

snpe = SNPE(prior=prior.get_joint_uniform(device=device), device=device, density_estimator="maf")
density_estimator = snpe.append_simulations(x, y).train()
posterior = snpe.build_posterior(density_estimator)

/home/micahtilton/github/likelihood-free-inference/.venv/lib/python3.12/site-packages/sbi/inference/trainers/npe/npe_base.py:157: UserWarning: Data x has device 'cpu'. Moving x to the data_device 'cuda'. Training will proceed on device 'cuda'.
  theta, x = validate_theta_and_x(
/home/micahtilton/github/likelihood-free-inference/.venv/lib/python3.12/site-packages/sbi/inference/trainers/npe/npe_base.py:157: UserWarning: Parameters theta has device 'cpu'. Moving theta to the data_device 'cuda'. Training will proceed on device 'cuda'.
  theta, x = validate_theta_and_x(


 Neural network successfully converged after 254 epochs.

In [54]:
theta_true, x_obs = dataset.generate_dataset(samples=1)
theta_true, x_obs = theta_true[-1], x_obs[-1]

theta_true = theta_true.to("cuda")
x_obs = x_obs.to("cuda")

print("Observed simulation output:", x_obs)

# posterior_cpu = snpe.build_posterior(density_estimator, sample_with="")
# Use the learned posterior to sample inferred parameters given the observed output
posterior_samples = posterior.sample((10000,), x=x_obs)
print("Posterior samples shape:", posterior_samples.shape)

# Compute a point estimate (e.g. the posterior mean)
posterior_mean = posterior_samples.mean(dim=0)
print("Posterior mean estimate:", posterior_mean)

print()
print("Theta True      :", theta_true)
print("Theta Prediction:", posterior_mean)

Observed simulation output: tensor([5.9058, 4.6106, 3.9709, 3.6128, 6.0883, 5.0537, 4.6149, 4.3983, 9.0000],
       device='cuda:0')


Drawing 10000 posterior samples: 11691it [00:00, 510500.43it/s]           

Posterior samples shape: torch.Size([10000, 2])
Posterior mean estimate: tensor([ 57.0861, 691.8786], device='cuda:0')

Theta True      : tensor([ 58.5589, 819.2974], device='cuda:0')
Theta Prediction: tensor([ 57.0861, 691.8786], device='cuda:0')
